# Log <small>global class</small>
Log implements a G-Set CRDT and adds ordering
Create a new Log instance

From:
"A comprehensive study of Convergent and Commutative Replicated Data Types"
https://hal.inria.fr/inria-00555588
* **ipfs**  IPFS An IPFS instance
* **access**  Object AccessController (./default-access-controller)
* **identity**  Object Identity (https://github.com/orbitdb/orbit-db-identity-provider/blob/master/src/identity.js)
* **logId**  String ID of the log
* **entries**  Array.<Entry> An Array of Entries from which to create the log
* **heads**  Array.<Entry> Set the heads of the log
* **clock**  Clock Set the clock of the log

**Returns:  Log** - Log

In [0]:
const IPFS = require("ipfs")
const Log = require("../src/log")
const { AccessController, IdentityProvider } = require("../src/log")
const Keystore = require('orbit-db-keystore')
const Entry = require("../src/entry")
const Clock = require('../src/lamport-clock')

const accessController = new AccessController()
const ipfs = new IPFS();
const keystore = Keystore.create("../test/fixtures/keys")
const identitySignerFn = async (id, data) => {
  const key = await keystore.getKey(id)
  return keystore.sign(key, data)
}

(async () => {
  var identity = await IdentityProvider.createIdentity(keystore, 'username', identitySignerFn)
  var log = new Log(ipfs, accessController, identity)

  // console.log(Object.keys(log))
})()

# Log#id <small>instance member</small>
Returns the ID of the log

**Returns:  string** - the ID of the log

In [0]:
(async () => {
  var identity = await IdentityProvider.createIdentity(keystore, 'username', identitySignerFn)
  var log = new Log(ipfs, accessController, identity)
  console.log(log.id) // default uses JS microtime

  var log2 = new Log(ipfs, accessController, identity, "MyLogID")
  console.log(log2.id) // or you can specify your own
})()

# Log#clock <small>instance member</small>
Returns the clock of the log

**Returns:  string** - The log's LamportClock

In [0]:
(async () => {
  var identity = await IdentityProvider.createIdentity(keystore, 'username', identitySignerFn)
  var log = new Log(ipfs, accessController, identity)
  console.log(log.clock)
})()

# Log#length <small>instance member</small>
Returns the length of the log

**Returns:  Number** - length of the log

In [0]:
(async () => {
  var identity = await IdentityProvider.createIdentity(keystore, 'username', identitySignerFn)
  var log = new Log(ipfs, accessController, identity)
  console.log(log.length)

  var entry = await Entry.create(ipfs, identity, '1', 'entry1', [], new Clock('1', 0))
  await log.append(entry)
  console.log(log.length)
})()

# Log#values <small>instance member</small>
Returns the values in the log

**Returns:  Array.<Entry>** - all values of the log sorted by LastWriteWins

In [0]:
(async () => {
  var identity = await IdentityProvider.createIdentity(keystore, 'username', identitySignerFn)
  var log = new Log(ipfs, accessController, identity)

  var entry = await Entry.create(ipfs, identity, '1', 'entry1', [], new Clock('1', 0))
  await log.append(entry)
  console.log(log.values.map(e => e.payload.payload)
})()

# Log#heads <small>instance member</small>
Returns an array of heads as multihashes

**Returns:  Array.<string>** - values of the log head(s)

In [0]:
(async () => {
  var identity1 = await IdentityProvider.createIdentity(keystore, 'username', identitySignerFn)
  var identity2 = await IdentityProvider.createIdentity(keystore, 'username', identitySignerFn)

  var log1 = new Log(ipfs, accessController, identity1)
  var log2 = new Log(ipfs, accessController, identity2)

  await log1.append("entryA1")
  await log1.append("entryA2")
  await log1.append("entryA3")

  await log2.append("entryB1")
  await log2.append("entryB2")

  await log1.join(log2)

  console.log(log1.heads.map(e => e.payload))
})()

# Log#tails <small>instance member</small>
**Note: May be deprecated**

Returns an array of Entry objects that reference entries which
are not in the log currently

**Returns:  Array.<Entry>** - values of the log tail(s)s

In [0]:
(async () => {
  var identity = await IdentityProvider.createIdentity(keystore, 'username', identitySignerFn)
  var log = new Log(ipfs, accessController, identity)
})()

# Log#tailHashes <small>instance member</small>
**Note: May be deprecated**

Returns an array of multihashes that are referenced by entries which
are not in the log currently

**Returns:  Array.<string>** - Array of multihashes

In [0]:
(async () => {
  var identity = await IdentityProvider.createIdentity(keystore, 'username', identitySignerFn)
  var log = new Log(ipfs, accessController, identity)
})()

# Log#get <small>instance function</small>
Find an entry
* **hash**  string The Multihash of the entry as Base58 encoded string

**Returns:  Entry undefined** - hash of the entry index

In [0]:
(async () => {
  var identity = await IdentityProvider.createIdentity(keystore, 'username', identitySignerFn)
  var log = new Log(ipfs, accessController, identity)

  await log.append("entry1")
  await log.append("entry2")
  await log.append("entry3")

  var hash = Object.keys(log._entryIndex)[1]
  console.log(log.get(hash).payload)
})()

# Log#has <small>instance function</small>
Verify that the log contains the entry you're seeking
* **entry**  Entry the entry you're looking to verify

**Returns:  Boolean** - `true` or `false` if the log contains the entry

In [0]:
(async () => {
  var identity = await IdentityProvider.createIdentity(keystore, 'username', identitySignerFn)
  var log = new Log(ipfs, accessController, identity)

  await log.append("entry1")
  await log.append("entry2")
  await log.append("entry3")

  var hash = Object.keys(log._entryIndex)[1]
  console.log(`${hash} --> ${log.has(hash)}`)
})()

# Log#traverse <small>instance function</small>
Follow the pointers and load the log into memory for processing
* **rootEntries**  Array entry or entries to start from
* **amount**  Number number of entries to traverse

**Returns:  Object** - object containing traversed entries

In [0]:
(async () => {
  var identity = await IdentityProvider.createIdentity(keystore, 'username', identitySignerFn)
  var log = new Log(ipfs, accessController, identity)

  await log.append("entryA1")
  await log.append("entryA2")
  await log.append("entryA3")

  var traverse1 = log.traverse(log.heads, 1)
  console.log(Object.keys(traverse1).length)

  var traverse2 = log.traverse(log.heads, 2)
  console.log(Object.keys(traverse2).length)

  var traverse3 = log.traverse(log.heads, 3)
  console.log(Object.keys(traverse3).length)
})()

# Log#append <small>instance function</small>
Append an entry to the log
* **data**  Entry Entry to add
* **pointerCount**  Number "Depth" of log to traverse

**Returns:  Log** - New Log containing the appended value

In [0]:
(async () => {
  var identity = await IdentityProvider.createIdentity(keystore, 'username', identitySignerFn)
  var log = new Log(ipfs, accessController, identity)

  await log.append("entry1")
  await log.append("entry2")
  await log.append("entry3")

  console.log(log.length)
 })()

# Log#join <small>instance function</small>
Joins two logs returning a new log. Doesn't mutate the original logs.
* **log**  Log Log to join with this Log
* **size**  Number Max size of the joined log

**Returns:  Promise.<Log>** - The promise of a new Log

In [0]:
(async () => {
  var identity1 = await IdentityProvider.createIdentity(keystore, 'username', identitySignerFn)
  var identity2 = await IdentityProvider.createIdentity(keystore, 'username', identitySignerFn)

  var log1 = new Log(ipfs, accessController, identity1)
  var log2 = new Log(ipfs, accessController, identity2)

  await log1.append("entryA1")
  await log1.append("entryA2")
  await log1.append("entryA3")

  await log2.append("entryB1")
  await log2.append("entryB2")

  await log1.join(log2)

  console.log(log1.length)
})()

# Log#toJSON <small>instance function</small>
Get the log in JSON format

**Returns:  Object.<{id, heads}>** - object with the id of the log and the heads

In [0]:
(async () => {
  var identity = await IdentityProvider.createIdentity(keystore, 'username', identitySignerFn)
  var log = new Log(ipfs, accessController, identity)

  await log.append("entryA1")
  await log.append("entryA2")
  await log.append("entryA3")

  console.log(log.toJSON())
})()

# Log#toSnapshot <small>instance function</small>
Get a snapshot of the log

**Returns:  Object.<{id, heads, values}>** - object with id, heads, and values array

In [0]:
(async () => {
  var identity = await IdentityProvider.createIdentity(keystore, 'username', identitySignerFn)
  var log = new Log(ipfs, accessController, identity)

  await log.append("entryA1")
  await log.append("entryA2")
  await log.append("entryA3")

  console.log(log.toSnapshot())
})()
(async () => {
  var identity = await IdentityProvider.createIdentity(keystore, 'username', identitySignerFn)
  var log = new Log(ipfs, accessController, identity)
})()

# Log#toBuffer <small>instance function</small>
Get the log as a Buffer

**Returns:  Buffer** - Buffer version of stringified log JSON

In [0]:
(async () => {
  var identity = await IdentityProvider.createIdentity(keystore, 'username', identitySignerFn)
  var log = new Log(ipfs, accessController, identity)

  await log.append("entryA1")
  await log.append("entryA2")
  await log.append("entryA3")

  console.log(log.toBuffer())
})()
(async () => {
  var identity = await IdentityProvider.createIdentity(keystore, 'username', identitySignerFn)
  var log = new Log(ipfs, accessController, identity)
})()

# Log#toString <small>instance function</small>
Returns the log entries as a formatted string
* **payloadMapper**  function transformation function

**Returns:  string** - plain text representation of the log

In [0]:
(async () => {
  var identity = await IdentityProvider.createIdentity(keystore, 'username', identitySignerFn)
  var log = new Log(ipfs, accessController, identity)

  await log.append("entryA1")
  await log.append("entryA2")
  await log.append("entryA3")

  console.log(log.toString())
})()

# Log.isLog <small>static function</small>
Check whether an object is a Log instance
* **log**  Object An object to check

**Returns:  true false** - true or false if the object is a log instance

In [11]:
(async () => {
  var identity = await IdentityProvider.createIdentity(keystore, 'username', identitySignerFn)
  var log = new Log(ipfs, accessController, identity)
  var notALog = { "Not": "Log"}
  
  console.log(Log.isLog(log))
  console.log(Log.isLog(notALog))
})()

true
false


# Log#toMultihash <small>instance function</small>
Get the log's multihash

**Returns:  Promise.<string>** - Multihash of the Log as Base58 encoded string

In [13]:
(async () => {
  var identity = await IdentityProvider.createIdentity(keystore, 'username', identitySignerFn)
  var log = new Log(ipfs, accessController, identity)

  await log.append("entryA1")
  await log.append("entryA2")
  await log.append("entryA3")

  console.log(await log.toMultihash())
})()

QmcPq5jYRRPDYdepora8D4xZHTZpYZ42cDzM6wpJRgk96z


# onProgressCallback <small>global typedef</small>
On Progress Callback
* **hash**  String 
* **entry**  Entry 
* **depth**  Number 

# Log.fromMultihash <small>static function</small>
Create a log from multihash
* **ipfs**  IPFS An IPFS instance
* **access**  AccessController AccessController object for the Log
* **identity**  Identity The identity of the owner of the log
* **hash**  string Multihash (as a Base58 encoded string) to create the log from
* **length**  Number [length=-1] How many items to include in the log
* **exclude**  Entry Entries to ex;lude from the log
* **onProgressCallback**  onProgressCallback On Progress Callback

**Returns:  Promise.<Log>** - New Log

In [15]:
(async () => {
  var identity = await IdentityProvider.createIdentity(keystore, 'username', identitySignerFn)
  var log = new Log(ipfs, accessController, identity)

  await log.append("entryA1")
  await log.append("entryA2")
  await log.append("entryA3")

  var multihash = await log.toMultihash();
  var logFromHash = await Log.fromMultihash(ipfs, accessController, identity, multihash)
      
  console.log(logFromHash.values.map(e => e.payload))
})()

[ 'entryA1', 'entryA2', 'entryA3' ]


# onProgressCallbackWithParent <small>global typedef</small>
On Progress Callback
* **hash**  String 
* **entry**  Entry 
* **parent**  Entry 
* **depth**  Number 

# Log.fromEntryHash <small>static function</small>
Create a log from a single entry's multihash
* **ipfs**  IPFS An IPFS instance
* **access**  AccessController AccessController instance for the log
* **identity**  Identity Identity object for the hash
* **hash**  string Multihash (as a Base58 encoded string) of the Entry from which to create the log from
* **id**  Number the ID of the new log
* **length**  Number How many entries to include in the log
* **exclude**  Array.<Entry> entries to exclude from the new log
* **onProgressCallback**  onProgressCallback On Progress Callback

**Returns:  Promise.<Log>** - New Log

In [0]:
(async () => {
  var identity = await IdentityProvider.createIdentity(keystore, 'username', identitySignerFn)
  var log = new Log(ipfs, accessController, identity)
})()

# Log.fromJSON <small>static function</small>
Create a log from a Log Snapshot JSON
* **ipfs**  IPFS An IPFS instance
* **access**  AccessController AccessController instance for the log
* **identity**  Identity Identity object for the hash
* **json**  Object Log snapshot as JSON object
* **length**  Number How many entries to include in the log
* **timeout**  Number number of milliseconds to time out in
* **onProgressCallback**  onProgressCallback On progress callback

**Returns:  Promise.<Log>** - New Log

In [0]:
(async () => {
  var identity = await IdentityProvider.createIdentity(keystore, 'username', identitySignerFn)
  var log = new Log(ipfs, accessController, identity)
})()

# Log.fromEntry <small>static function</small>
Create a new log from an Entry instance
* **ipfs**  IPFS An IPFS instance
* **access**  AccessController AccessController instance for the log
* **identity**  Identity Identity object for the hash
* **sourceEntries**  Entry Array.<Entry> An Entry or an array of entries to fetch a log from
* **length**  Number How many entries to include. Default: infinite.
* **exclude**  Array.<(Entry|string)> Array of entries or hashes or entries to not fetch (foe eg. cached entries)
* **onProgressCallback**  onProgressCallback On progress callback

**Returns:  Promise.<Log>** - New Log

In [0]:
(async () => {
  var identity = await IdentityProvider.createIdentity(keystore, 'username', identitySignerFn)
  var log = new Log(ipfs, accessController, identity)
})()

# Log.findHeads <small>static function</small>
Finds entries that are the heads of this collection,
ie. entries that are not referenced by other entries
* **entries**  Array.<Entry> Entries to search heads from

**Returns:  Array.<Entry>** - entryHash

In [0]:
(async () => {
  var identity = await IdentityProvider.createIdentity(keystore, 'username', identitySignerFn)
  var log = new Log(ipfs, accessController, identity)
})()

# Log.findTails <small>static function</small>
Find entries that point to another entry that is not in the input array
* **entries**  Array.<Entry> entried to find tails from

**Returns:  Array.<Entry>** - unique tail entries

In [0]:
(async () => {
  var identity = await IdentityProvider.createIdentity(keystore, 'username', identitySignerFn)
  var log = new Log(ipfs, accessController, identity)
})()

# Log.findTailHashes <small>static function</small>
Find the hashes to entries that are not in a collection
but referenced by other entries
* **entries**  Array.<Entry> array of entries to find tails in

**Returns:  Array.<String>** - hashes of tail entries

In [0]:
(async () => {
  var identity = await IdentityProvider.createIdentity(keystore, 'username', identitySignerFn)
  var log = new Log(ipfs, accessController, identity)
})()

# Log.difference <small>static function</small>
Shows the difference between two logs
* **a**  Log the first log
* **b**  Log the second log

**Returns:  Log** - The resultant log

In [0]:
(async () => {
  var identity = await IdentityProvider.createIdentity(keystore, 'username', identitySignerFn)
  var log = new Log(ipfs, accessController, identity)
})()